In [13]:
# user defined parameters
targeted_portfolio_names_file = 'targeted_portfolio_names.txt'
bid_multiplier_factor = 0.9
#file_name = 'bulk-a3l8g1tgo8s0og-20230105-20230106-1673016841890.xlsx'
#file_name = 'duptest.xlsx'
file_name = 'no_empty_bids_test.xlsx'

In [14]:
SHEET_NAME_0 = 'Portfolios'
SHEET_NAME_1 = 'Sponsored Products Campaigns'
SHEET_NAME_2 = 'Sponsored Brands Campaigns'
SHEET_NAME_3 = 'Sponsored Display Campaigns'
entity_type_filter = {'Keyword', 'Product Targeting'}
ENABLED = 'enabled'
UPDATE = 'Update'

In [15]:
# make sure these packages are installed
import pandas as pd
import numpy as np
from datetime import datetime

In [16]:
# Python version of this notebook
from platform import python_version
print(python_version())

3.8.16


In [18]:
targeted_portfolio_name_list = pd.read_csv(targeted_portfolio_names_file, sep=',', header=None)[0].tolist()
targeted_portfolio_names = set(targeted_portfolio_name_list)

In [19]:
targeted_portfolio_names

{'00A96 Rain Poncho ACOS25',
 '03013 OxfordMoving ACOS30',
 'asd',
 'csada',
 'csdsa',
 'sdas'}

In [20]:
df_portfolios = pd.read_excel(file_name, SHEET_NAME_0)

In [21]:
df_portfolios.dtypes

Product                            object
Entity                             object
Operation                         float64
Portfolio Id                        int64
Portfolio Name                     object
Budget Amount                     float64
Budget Currency Code               object
Budget Policy                     float64
Budget Start Date                 float64
Budget End Date                   float64
State (Informational only)         object
In Budget (Informational only)       bool
dtype: object

In [22]:
# check duplicates in portfolio names.
if not df_portfolios['Portfolio Name'].is_unique:
    duplicate_portfolio_names = set()
    portfolio_names = set()
    
    for name in df_portfolios['Portfolio Name']:
        if name in portfolio_names:
            duplicate_portfolio_names.add(name)
        portfolio_names.add(name)
        
    raise Exception("Duplicates are found in the following portfolio names: " 
                        + '; '.join(duplicate_portfolio_names)
                       + '\nPlease eliminate duplicates before proceeding again. ')

print("No duplicates are found in portfolio names. You may proceed!")

No duplicates are found in portfolio names. You may proceed!


In [23]:
def process_spreadsheet(file_name, sheet_name):
    campaign_df = pd.read_excel(file_name, sheet_name)
    
    campaign_df['Operation'] = campaign_df['Operation'].astype(str)
    
    campaign_df_filtered = campaign_df.loc[(campaign_df['Portfolio Name (Informational only)'].isin(targeted_portfolio_names))
                                         & (campaign_df['Entity'].isin(entity_type_filter))
                                         & (campaign_df['State'] == ENABLED)
                                         & (campaign_df['Campaign State (Informational only)'] == ENABLED)
                                         & (campaign_df['Ad Group State (Informational only)'] == ENABLED)]
    if campaign_df_filtered['Bid'].isnull().sum() > 0:
        raise Exception("Empty bids are found in the following rows: " 
                        + '; '.join(str(x) for x in campaign_df_filtered[campaign_df_filtered['Bid'].isnull()].index.tolist())
                       + '\nPlease eliminate empty bids before proceeding again. ')
    
    campaign_df['Operation'] = np.where(
        ((campaign_df['Portfolio Name (Informational only)'].isin(targeted_portfolio_names)) 
       & (campaign_df['Entity'].isin(entity_type_filter))
       & (campaign_df['State'] == ENABLED)
       & (campaign_df['Campaign State (Informational only)'] == ENABLED)
       & (campaign_df['Ad Group State (Informational only)'] == ENABLED)    
        ),
        UPDATE,
        '')
    
    campaign_df['Bid'] = np.where(
        (campaign_df['Operation'] == UPDATE),
        campaign_df['Bid'] * bid_multiplier_factor,
        campaign_df['Bid'])
    
    current_dateTime = datetime.utcnow().__str__().split('.')[0].replace(' ', '_').replace(':', '#')
    campaign_df.to_excel(sheet_name.replace(' ', '_') + "_" + current_dateTime + "_" + file_name, 
                    sheet_name=sheet_name, 
                    index=False)  
    
    print('COMPLETED!')

In [12]:
process_spreadsheet(file_name, SHEET_NAME_1)

COMPLETED!
